# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


импорт библиотек

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

опции вывода данных

In [3]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [4]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/sport/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',         
]

выбираем начальные страницы для парсинга

In [5]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'url', 'date', 'availability', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'url' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'date' : pd.to_datetime('today').normalize(),
                             'availability' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

Создаем функцию предварительного парсинга

179  -  16  -  23  -  100  -  33  -  60  -  10  -  4  -  31  -  25  -  5  -  5  -  81 

In [6]:
dft = parsing_2(oll_url)
dft

  0%|          | 0/187 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

,title,url,date,availability,id,price,old_price
0,"Планшет графич. Parblo Intangbo X7 (8192 ур. нажатия, 5080lpi, раб. область 189x114мм, беспр.перо S01, USB)",https://tiraet.com/catalog/Komplektujuschie/kompjutery/planshety/planshet_grafich_parblo_intangbo_x7_8192_ur_nazhatiya_5080lpi_rab_oblast_189x114mm_bespr_pero_s01_us/,2024-08-06,Достаточно,88081,NaN,NaN
1,Адаптер (салазки для жестких дисков HP 2.5' SAS/SATA) SATA3 (651687-001),https://tiraet.com/catalog/Komplektujuschie/komplektujuschie/Karmany-HDD/adapter_salazki_dlya_zhestkikh_diskov_hp_2_5_sas_sata_sata3_651687_001/,2024-08-06,Достаточно,88076,NaN,NaN
2,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/Perehodnik-dlja-myshi-COM-v-PS2-A-235/,2024-08-06,Много,284,5.0,NaN
3,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-USB2-AM4P-6)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/kabel-mini-usb20-a---mini-usb-4pm18m-cc-usb2-am4p-6/,2024-08-06,Много,8683,5.0,NaN
4,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-08-06,Много,85078,6.6,NaN
...,...,...,...,...,...,...,...
11686,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-08-06,Мало,27477,7920.0,NaN
11687,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-08-06,Мало,55309,8085.0,NaN
11688,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-08-06,Мало,56144,8250.0,NaN
11689,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-08-06,Нет в наличии,249,38.0,NaN


dft.to_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

dft = pd.read_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

тестовое сохранение информации после предварительного парсинга

In [7]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv')

Загружаем данные из файла  с информацией о группах и категориях товара по его `id`

In [8]:
tiraet_group = tiraet_group.drop_duplicates(keep= False )

Удаляем дублекаты в загруженном фрейме

In [9]:
dft = dft.merge(tiraet_group, how='left', on='id')

Объединяем датафрейм предварительного парсинга с датафреймом о группах и категориях товара

In [10]:
t_df_new = dft[(dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna())]

Отфильтровываем в отдельный датафрейм данные не содержащие информацию о группах и категориях товара

In [11]:
dft = dft[~((dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna()))]

Отчищаем основной датафрейм от данных о товарах не содержащих информацию о группах и категориях

In [12]:
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            else:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_2[4].text.replace(',','').replace(' ','_').replace('-','_').lower())
        else:
            d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_16 = 'T_'+(elem_1[2].text.replace(',','').replace(' ','_').replace('-','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

Создаем функцию для парсинга данных о группах и категориях товара

In [13]:
tqdm.pandas() 
t_df_new[['_category','_subcategory','_group']] = t_df_new['url'].progress_apply(find_groups)

  0%|          | 0/44 [00:00<?, ?it/s]

Парсим данные о группах и категориях товара

In [14]:
t_df_new._category.unique()

array(['T_компьютерная_техника', 'err_', 'T_бытовая_техника',
       'T_светотехника'], dtype=object)

проссматриваем информацию о категориях нового товара

In [15]:
t_df_new._subcategory.unique()

array(['T_комплектующие', 'T_аксессуары_компьютерные', 'T_мониторы',
       'T_мультимедиа', 'err_', 'T_телевизоры', 'T_уличное_освещение',
       'T_оборудование_электромонтажное'], dtype=object)

Просматриваем информацию о подкатегориях нового товара

In [16]:
t_df_new._group.unique()

array(['T_приводы_оптические', 'T_диски_дискеты', 'T_мониторы',
       'T_саундбары', 'err_', 'T_телевизоры',
       'T_светильники_садово_парковые', 'T_выключатели_розетки',
       'T_коробки_клеммы'], dtype=object)

Просматриваем информацию о группах нового товара

In [17]:
t_data = t_df_new[~(t_df_new['_category']=='err_')].reset_index(drop=True)
t_data = t_data[['id', '_category','_subcategory','_group' ]]
t_data = t_data.sort_values(['_category','_subcategory', '_group'])

Выделяем информацию о группах и категориях нового товара в отдельную переменную и сортируем ее

In [18]:
tiraet_group = pd.concat([tiraet_group, t_data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False)  

Обединяем информацию о группах и категориях товара в один датафрейм и сохраняем ее в старый файл

In [19]:
dft = pd.concat([dft, t_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

Объединяем очищенный датафрейм с информацией о товарах и группах товаровс новым датафреймом о товарах и группах товаров

dft.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

тестовое сохранение полученной информации

In [20]:
dft[dft['_category']=='err_'].id.count()

1

смотрим количество данных спарсенное с ошибкой

In [21]:
dft[dft['_category']=='err_']

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group
11690,"Музыкальный центр PANASONIC SC-AKX320GSK (2.0, 450Вт, CD, FM, USB, SD, BT, Д/У) BLACK",https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/muzykalnye-centry/muzykalnyy_tsentr_panasonic_sc_akx320gsk_2_0_450vt_cd_fm_usb_sd_bt_d_u_black/,2024-08-06,Мало,87154,3799.0,NaN,err_,err_,err_


просматриваем данные спарсенные с ошибкой

In [22]:
dft['error_group'] = 0
dft.loc[dft['_category']=='err', 'error_group'] = 1

В датафрейме создаем столбец данные в котором равны "1", если данные о группе товаров спарсены с ошибкой

In [23]:
for i in dft['_category'].unique():
    t = dft[dft['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

T_автотовары
['T_автомагнитола' 'T_автопылесос' 'T_авторегистратор' 'T_автохолодильник'
 'T_динамики' 'T_компрессоры' 'T_крепления_смартфонов/планшетов'
 'T_модуляторы_fm' 'T_навигаторы_gps']
----------------------------------------------------------------------------------------------------
T_безопасность
['T_видеонаблюдение' 'T_домофоны' 'T_монтаж_и_питание'
 'T_оборудование_poe' 'T_сигнализация_охранно_пожарная' 'T_скуд'
 'T_умный_дом']
----------------------------------------------------------------------------------------------------
T_бытовая_техника
['T_гигиена_здоровье_уход' 'T_климатическая_техника'
 'T_крупная_бытовая_техника' 'T_мелкая_бытовая_техника' 'T_телевизоры'
 'T_техника_для_кухни' 'T_часы_наручные']
----------------------------------------------------------------------------------------------------
T_для_дома
['T_бытовая_химия' 'T_запасные_части' 'T_инструменты' 'T_крепления'
 'T_кухонная_утварь' 'T_мебель' 'T_текстиль' 'T_хозяйственный_инвентарь']
-----------------

# Аналитика акционных товаров

In [24]:
brend = pd.read_excel('csv/brend.xlsx')

In [25]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [26]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
dft['brend'] = dft['title'].progress_apply(brends)

  0%|          | 0/11691 [00:00<?, ?it/s]

создаем столбец с брендами

In [27]:
dft['action_sale'] = 0
dft.loc[dft['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [28]:
dft['sale'] = (100 - ((dft['price']*100)/dft['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [29]:
def count_group(x):
    return dft[dft['_group']==x]['id'].count()    
tqdm.pandas()
dft['count_group'] = dft['_group'].progress_apply(count_group)

  0%|          | 0/11691 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [30]:
def count_group_presence(x):
    return dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['id'].count()   
tqdm.pandas()
dft['activ_count_group'] = dft['_group'].progress_apply(count_group_presence)

  0%|          | 0/11691 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [31]:
def dif_price(x):    
    max_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].max()
    min_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dft['dif_price'] = dft['_group'].progress_apply(dif_price)

  0%|          | 0/11691 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [32]:
dft['group_prise'] = dft.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [33]:
dft = dft[['url', 'id', '_category', '_subcategory', '_group', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group', 'error_group']]

In [34]:
dft['tir'] = 1

In [35]:
dft = dft.drop_duplicates().reset_index(drop=True)

In [36]:
preprocessing(dft)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_navitel_rd5_4x50vt_msd_2_usb_bluetooth/,80126,T_автотовары,T_автомагнитола,T_автомагнитола,navitel,"Автомагнитола NAVITEL RD5 (4x50Вт, mSD, 2*USB, Bluetooth)",2024-08-06,660.0,NaN,0,NaN,бюджет,3.331818,Достаточно,9,9,0,1
1,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",2024-08-06,660.0,NaN,0,NaN,бюджет,3.331818,Мало,9,9,0,1
2,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_akai_ca016a_9008u_bluetooth_fm_am_mp3_usb_sd_aux_4_25w_1din_black_4905192533076/,83899,T_автотовары,T_автомагнитола,T_автомагнитола,akai,"Автомагнитола AKAI CA016A-9008U (Bluetooth, FM/AM, MP3, USB/SD, AUX, 4*25W,1DIN) black 4905192533076",2024-08-06,759.0,NaN,0,NaN,бюджет,3.331818,Достаточно,9,9,0,1
3,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",2024-08-06,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,9,9,0,1
4,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola-pioneer-mvh-29bt-4x50vt-mp3wmawavflac-fmam-usb-1-din-bluetooth-isorca/,63822,T_автотовары,T_автомагнитола,T_автомагнитола,pioneer,"Автомагнитола Pioneer MVH-29BT 4x50Вт, MP3/WMA/WAV/FLAC, FM/AM, USB, 1 DIN, Bluetooth, ISO/RCA",2024-08-06,1699.0,NaN,0,NaN,дорогой,3.331818,Достаточно,9,9,0,1


****************************************************************************************************
таблица имеет 11689 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                   15
old_price            10164
action_sale              0
sale                 10164
group_prise             15
dif_price                2
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.1
old_price            87.0
action_sale           0.0
sale                 87.0
group_prise           0.1
dif_price             0.0
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11689 entries, 0 to 11688
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                11689 non-null  object        
 1   id                 11689 non-null  object        
 2   _category          11689 non-null  object        
 3   _subcategory       11689 non-null  object        
 4   _group             11689 non-null  object        
 5   brend              11689 non-null  object        
 6   title              11689 non-null  object        
 7   date               11689 non-null  datetime64[ns]
 8   price              11674 non-null  float64   

None

# Парсинг Hi

In [37]:
def h_find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [38]:
def h_find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [39]:
def h_find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [40]:
def h_find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [41]:
def h_find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [42]:
def h_find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [43]:
def h_parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        attempt = 1
        while q>0 or attempt<4:             
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
                attempt = 1                
            else:
                if attempt == 3:                    
                    q = 0
                    attempt += 1
                else:
                    attempt += 1                    
    data['id'] = data['url'].apply(h_find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(h_find_availability)
    data['title'] = data['url'].apply(h_find_title)
    data['price'] = data['url'].apply(h_find_price)
    data['old_price'] = data['url'].apply(h_find_old_price)
    data['url'] = data['url'].apply(h_find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [44]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

переменная со списком ссылок на сайт для парсинга

In [45]:
dfh = h_parsing_2(all_url)

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [46]:
dfh = dfh.drop_duplicates().reset_index(drop=True)

удаление дублекатов

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [47]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-08-06,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-08-06,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 9178 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       7018
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       76.5
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9178 entries, 0 to 9177
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9178 non-null   object        
 1   id            9178 non-null   object        
 2   date          9178 non-null   datetime64[ns]
 3   availability  9178 non-null   object        
 4   title         9178 non-null   object        
 5   price         9178 non-null   float64       
 6   old_price     2160 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 502.1+ KB


None

In [48]:
hai_group = pd.read_csv('csv/ahay_group.csv')

загрузка таблицы с группами товара

In [49]:
hai_group = hai_group.drop_duplicates(keep=False )

In [50]:
len(hai_group)

10164

In [51]:
dfh = dfh.merge(hai_group, how='left', on='id')

обединение таблиц предварительного парсинга и таблицы групп товаров

In [52]:
dfh

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-08-06,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-08-06,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-08-06,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
...,...,...,...,...,...,...,...,...,...,...,...
9173,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-transparent-led15w-e27-5628-tkl/,Т-000085036,2024-08-06,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628) (TKL),3279.0,4099.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9174,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-amber-led15w-e27-5625-tkl/,Т-000085037,2024-08-06,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9175,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-3pl-universo-black-led15w-e27-4483-tkl/,Т-000085021,2024-08-06,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9176,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-6pl-fairy-black-6w-g9-4856-tkl/,Т-000085033,2024-08-06,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры


In [53]:
h_df_new = dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())]

создание тыблицы с товарами, у которых нет данных о группах

In [54]:
len(h_df_new)

35

In [55]:
dfh = dfh[~((dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna()))]

отчистка таблицы парсинга от пропусков

In [56]:
len(dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())])

0

dfh = dfh.sample(30)

тестовое уменьшение таблици

In [57]:
def h_xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг групп товара

In [58]:
if len(h_df_new) > 0:
    tqdm.pandas() 
    h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)
else:
    pass

  0%|          | 0/35 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_12536\1426536857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)


Основной парсинг

In [59]:
h_data = h_df_new[~((h_df_new['_category']=='err1') | (h_df_new['_category']=='err0'))].reset_index(drop=True)
h_data = h_data[['id', '_category','_subcategory','_group', '_subgroup']]
h_data = h_data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

In [60]:
len(h_data)

32

создание таблици с группами нового товара

In [61]:
hai_group = pd.concat([hai_group, h_data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

добавление в таблицу с группами товара новых данных

In [62]:
dfh = pd.concat([dfh, h_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

объединение таблици со старыми и новыми товарами

In [63]:
dfh._category.unique()

array(['H_бытовая_техника', 'H_компьютерная_техника', 'H_мебель_текстиль',
       'H_по_умолчанию', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_товар_без_акции',
       'H_товар_по_акции', 'H_товары_для_дома', 'err1'], dtype=object)

In [64]:
dfh._subcategory.unique()

array(['H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_apple', 'H_аксессуары',
       'H_видеотехника', 'H_готовые_решения', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_мультимедиа',
       'H_накопители', 'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_умный_дом_с_яндекс',
       'H_электропитание', 'H_мебель_для_офиса', 'H_по_умолчанию',
       'H_техника_для_дома', 'H_электроинструменты', 'H_аудиосистемы',
       'H_батарейки_и_зарядные_устройства', 'H_гаджеты',
    

In [65]:
dfh._group.unique()

array(['H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_блендеры',
       'H_вакууматоры', 'H_весы_кухонные', 'H_генераторы_льда',
       'H_кухонные_комбайны', 'H_ломтерезки', 'H_миксеры',
       'H_соковыжималки', 'H_соковыжималки_для_цитрусов', 'H_тостеры',
       'H_чопперы', 'H_электромясорубки', 'H_минимойки', 'H_кофе',
       'H_кофеварки', 'H_кофемашины', 'H_кофемолки',

просмотр категорий, подкатегорий и групп товаров

In [66]:
dfh[dfh['_category']=='err1'].id.count()

3

In [67]:
dfh[dfh['_category']=='err0'].id.count()

0

In [68]:
dfh[dfh['_category']=='err1']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
9175,https://hi-tech.md/kompyuternaya-tehnika/komplektuyuschie/korpusa/korpus-natec-helix-npc-2038/,Т-000094407,2024-08-06,Ожидается,Корпус Natec HELIX (NPC-2038),749.0,NaN,err1,err1,err1,err1
9176,https://hi-tech.md/tovary-dlya-doma/kuhonnye-prinadlezhnosti/kuhonnye-aksessuary/sito-vicko-1246-00001-26sm/,Т-000094672,2024-08-06,Ожидается,сито Vicko 1246-00001 26см,56.0,NaN,err1,err1,err1,err1
9177,https://hi-tech.md/tovary-dlya-doma/prigotovlenie-pischi/soteyniki/soteynik-zilan-zln2089-28sm-antiprig.-f/,Т-000094792,2024-08-06,В наличии,сотейник Zilan ZLN2089 28см антиприг. ф,187.0,NaN,err1,err1,err1,err1


In [69]:
dfh[dfh['_category']=='err0']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup


просмотр товаров с ошибками

In [70]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-08-06,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,2024-08-06,В наличии,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,2024-08-06,В наличии,насадка д/Зубная щетка Braun EB18 3D White (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,2024-08-06,В наличии,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zubnaya-schetka-braun-eb50-5/,Т-000070385,2024-08-06,В наличии,насадка д/Зубная щетка Braun EB50 Cross Action White (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


****************************************************************************************************
таблица имеет 9178 сторок и 11 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       7018
_category          0
_subcategory       0
_group             0
_subgroup          0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       76.5
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
_category               object
_subcategory            object
_group                  object
_subgroup               object
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9178 entries, 0 to 9177
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9178 non-null   object        
 1   id            9178 non-null   object        
 2   date          9178 non-null   datetime64[ns]
 3   availability  9178 non-null   object        
 4   title         9178 non-null   object        
 5   price         9178 non-null   float64       
 6   old_price     2160 non-null   float64       
 7   _category     9178 non-null   object        
 8   _subcategory  9178 non-null   object        
 9   _group        9178 non-null   object        
 1

None

просмотр данных о таблице

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [71]:
dfh.head(2)

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-08-06,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,2024-08-06,В наличии,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


In [72]:
dfh['error_group'] = 0
dfh.loc[dfh['_subgroup'].isna(), 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err0', 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [73]:
dfh.loc[dfh['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [74]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

изменение порядка располажения столбцов

In [75]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-08-06,34.0,68.0,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-08-06,94.0,NaN,В наличии,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-08-06,94.0,NaN,В наличии,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-08-06,94.0,NaN,В наличии,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zubnaya-schetka-braun-eb50-5/,Т-000070385,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB50 Cross Action White (1шт),2024-08-06,94.0,NaN,В наличии,0


****************************************************************************************************
таблица имеет 9178 сторок и 12 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
_category          0
_subcategory       0
_group             0
_subgroup          0
title              0
date               0
price              0
old_price       7018
availability       0
error_group        0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
title            0.0
date             0.0
price            0.0
old_price       76.5
availability     0.0
error_group      0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
_category               object
_subcategory            object
_group                  object
_subgroup               object
title                   object
date            datetime64[ns]
price                  float64
old_price              float64
availability            object
error_group              int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9178 entries, 0 to 9177
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9178 non-null   object        
 1   id            9178 non-null   object        
 2   _category     9178 non-null   object        
 3   _subcategory  9178 non-null   object        
 4   _group        9178 non-null   object        
 5   _subgroup     9178 non-null   object        
 6   title         9178 non-null   object        
 7   date          9178 non-null   datetime64[ns]
 8   price         9178 non-null   float64       
 9   old_price     2160 non-null   float64       
 1

None

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [76]:
dfh.loc[dfh['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
dfh.loc[dfh['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_airpods', '_group'] = 'H_наушники'
dfh.loc[dfh['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
dfh.loc[dfh['_group']=='H_iphone', '_group'] = 'H_смартфоны'
dfh.loc[dfh['_group']=='H_ipad', '_group'] = 'H_планшеты'
dfh.loc[dfh['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
dfh.loc[dfh['_group']=='H_imac', '_group'] = 'H_моноблоки'
dfh.loc[dfh['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
dfh.loc[dfh['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
dfh.loc[dfh['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
dfh.loc[dfh['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
dfh.loc[dfh['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
dfh.loc[dfh['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
dfh.loc[dfh['_subgroup']=='H_замки', '_group'] = 'H_замки'
dfh.loc[dfh['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
dfh.loc[dfh['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
dfh.loc[dfh['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
dfh.loc[dfh['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [77]:
for i in dfh['_category'].unique():
    t = dfh[dfh['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

H_бытовая_техника
['H_аксессуары_для_мелкой_бытовой_техники' 'H_климатическая_техника'
 'H_крупная_техника_для_кухни' 'H_мелкая_техника_для_кухни' 'H_минимойки'
 'H_приготовление_напитков' 'H_приготовление_пищи'
 'H_пылесосы_и_аксессуары' 'H_стиральные_и_сушильные_машины'
 'H_техника_karcher' 'H_техника_для_здоровья' 'H_техника_для_красоты'
 'H_товары_для_ухода_за_одеждой' 'H_установка_бытовой_техники']
----------------------------------------------------------------------------------------------------
H_компьютерная_техника
['H_аудиосистемы' 'H_гаджеты' 'H_apple' 'H_готовые_решения'
 'H_смартфоны_и_аксессуары' 'H_аксессуары' 'H_видеотехника'
 'H_игровые_приставки' 'H_комплектующие' 'H_крепления' 'H_мониторы'
 'H_накопители' 'H_оргтехника' 'H_периферийные_устройства'
 'H_проекторы_и_экраны' 'H_расходные_материалы' 'H_сетевое_оборудование'
 'H_электропитание']
----------------------------------------------------------------------------------------------------
H_мебель_текстиль
['H_мебел

In [78]:
for i in dfh['_subcategory'].unique():
    t = dfh[dfh['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

H_аксессуары_для_мелкой_бытовой_техники
['H_аксессуары_для_зубных_щеток' 'H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_климатическая_техника
['H_бойлеры' 'H_вентиляторы' 'H_комплектующие_для_климатической_техники'
 'H_кондиционеры' 'H_кондиционеры_кассетные' 'H_кондиционеры_мульти_сплит'
 'H_кондиционеры_промышленные' 'H_котлы_газовые' 'H_обогреватели'
 'H_отопительные_печи' 'H_увлажнители_комплексы']
----------------------------------------------------------------------------------------------------
H_крупная_техника_для_кухни
['H_варочные_поверхности' 'H_вытяжки' 'H_духовые_шкафы'
 'H_кронштейны_для_микроволновых_печей' 'H_микроволновые_печи'
 'H_морозильники_и_лари' 'H_плиты_газовые_и_электрические'
 'H_посудомоечные_машины' 'H_холодильники' 'H_электропечи_и_духовки']
----------------------------------------------------------------------------------------------------
H_мелкая_техника_для

просмотр групп товаров в подкатегориях

In [79]:
for i in dfh['_group'].unique():
    t = dfh[dfh['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

H_аксессуары_для_зубных_щеток
['H_аксессуары_для_зубных_щеток']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_электрических_бритв
['H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_бойлеры
['H_бойлеры']
----------------------------------------------------------------------------------------------------
H_вентиляторы
['H_вентиляторы']
----------------------------------------------------------------------------------------------------
H_комплектующие_для_климатической_техники
['H_комплектующие_для_климатической_техники']
----------------------------------------------------------------------------------------------------
H_кондиционеры
['H_кондиционеры']
----------------------------------------------------------------------------------------------------
H_кондиционеры_кассетные
['H_кондиционеры_кассетные']
--------------------

H_электронные_книги
['H_электронные_книги']
----------------------------------------------------------------------------------------------------
H_геймпады
['H_геймпады']
----------------------------------------------------------------------------------------------------
H_приставки
['H_приставки']
----------------------------------------------------------------------------------------------------
H_блоки_питания
['H_блоки_питания']
----------------------------------------------------------------------------------------------------
H_видеокарты
['H_nvidia' 'H_radeon']
----------------------------------------------------------------------------------------------------
H_звуковые_карты
['H_звуковые_карты']
----------------------------------------------------------------------------------------------------
H_корпуса
['H_корпуса']
----------------------------------------------------------------------------------------------------
H_материнские_платы
['H_amd' 'H_intel' 'H_материнские_платы'

H_турки
['H_турки']
----------------------------------------------------------------------------------------------------
H_фильтры_для_воды
['H_фильтры_для_воды']
----------------------------------------------------------------------------------------------------
H_чайники_для_плиты
['H_чайники_для_плиты']
----------------------------------------------------------------------------------------------------
H_кастрюли
['H_кастрюли']
----------------------------------------------------------------------------------------------------
H_ковши
['H_ковши']
----------------------------------------------------------------------------------------------------
H_крышки
['H_крышки']
----------------------------------------------------------------------------------------------------
H_наборы_посуды
['H_наборы_посуды']
----------------------------------------------------------------------------------------------------
H_сковороды
['H_сковороды']
-------------------------------------------------------

In [80]:
dfh._category.unique()

array(['H_бытовая_техника', 'H_компьютерная_техника', 'H_мебель_текстиль',
       'H_по_умолчанию', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_no_group', 'H_товары_для_дома'],
      dtype=object)

In [81]:
dfh._subcategory.unique()

array(['H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_аудиосистемы', 'H_гаджеты',
       'H_apple', 'H_готовые_решения', 'H_смартфоны_и_аксессуары',
       'H_аксессуары', 'H_видеотехника', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_накопители',
       'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_электропитание', 'H_мебель_для_офиса',
       'H_по_умолчанию', 'H_техника_для_дома', 'H_электроинструменты',
       'H_батарейки_и_зарядные_устройства', 'H_крепления_для_

In [82]:
dfh._group.unique()

array(['H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_блендеры',
       'H_вакууматоры', 'H_весы_кухонные', 'H_генераторы_льда',
       'H_кухонные_комбайны', 'H_ломтерезки', 'H_миксеры',
       'H_соковыжималки', 'H_соковыжималки_для_цитрусов', 'H_тостеры',
       'H_чопперы', 'H_электромясорубки', 'H_минимойки', 'H_кофе',
       'H_кофеварки', 'H_кофемашины', 'H_кофемолки',

In [83]:
dfh._subgroup.unique()

array(['H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники', 'H_морозильные_лари',
       'H_газовые_плиты', 'H_комбинированные_плиты',
       'H_плиты_газовые_и_электрические', 'H_электрические_плиты',
       'H_посудомоечные_машины', 'H_side_by_side_и_french_door',
       'H_встраиваемые_холодильники', 'H_однодверные_холодильники',
       'H_с_верхней_морозильной_камерой',
       'H_с_нижней_морозильной_камерой', 'H_холодильники',
       'H_широкие_холодильники', 'H_электропечи_и_духовки',
       'H_блендеры

# Подготовка анализа

In [84]:
dfh.head(2)

,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-08-06,34.0,68.0,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-08-06,94.0,NaN,В наличии,0


def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

In [85]:
tqdm.pandas()
dfh['brend'] = dfh['title'].progress_apply(brends)

  0%|          | 0/9178 [00:00<?, ?it/s]

создаем столбец с брендами

In [86]:
dfh['action_sale'] = 0
dfh.loc[dfh['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [87]:
dfh['sale'] = (100 - ((dfh['price']*100)/dfh['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [88]:
def h_count_group(x):
    return dfh[dfh['_group']==x]['id'].count()    
tqdm.pandas()
dfh['count_group'] = dfh['_group'].progress_apply(h_count_group)

  0%|          | 0/9178 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [89]:
def h_count_group_presence(x):
    return dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['id'].count()   
tqdm.pandas()
dfh['activ_count_group'] = dfh['_group'].progress_apply(h_count_group_presence)

  0%|          | 0/9178 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [90]:
def h_dif_price(x):    
    max_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].max()
    min_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dfh['dif_price'] = dfh['_group'].progress_apply(h_dif_price)

  0%|          | 0/9178 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [91]:
dfh['group_prise'] = dfh.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [92]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group','error_group']]

In [93]:
dfh.drop('_subgroup', axis= 1 , inplace= True )

In [94]:
dfh['tir'] = 0

# объединение

In [95]:
df = pd.concat([dfh, dft], ignore_index=True)

In [96]:
df = df.drop_duplicates().reset_index(drop=True)

In [97]:
df.to_csv('csv/hay_tir_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [98]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,panasonic,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-08-06,34.0,68.0,1,50.0,бюджет,2.764706,В наличии,7,7,0,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-08-06,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-08-06,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-08-06,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zubnaya-schetka-braun-eb50-5/,Т-000070385,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB50 Cross Action White (1шт),2024-08-06,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0


****************************************************************************************************
таблица имеет 20867 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                   15
old_price            17182
action_sale              0
sale                 17182
group_prise             15
dif_price               54
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.1
old_price            82.3
action_sale           0.0
sale                 82.3
group_prise           0.1
dif_price             0.3
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20867 entries, 0 to 20866
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                20867 non-null  object        
 1   id                 20867 non-null  object        
 2   _category          20867 non-null  object        
 3   _subcategory       20867 non-null  object        
 4   _group             20867 non-null  object        
 5   brend              20867 non-null  object        
 6   title              20867 non-null  object        
 7   date               20867 non-null  datetime64[ns]
 8   price              20852 non-null  float64   

None

In [99]:
t = df[df['tir']==0]

In [100]:
len(t)

9178

In [101]:
t.sort_values('sale', ascending=False).head(100)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
5357,https://hi-tech.md/kompyuternaya-tehnika/elektropitanie/zaryadnye-usb/nfc/besprovodnaya-zaryadka-acme-ch302/,Т-000066021,H_компьютерная_техника,H_электропитание,H_зарядные_устройства,acme,Беспроводная зарядка Acme CH302,2024-08-06,99.0,199.0,1,50.25,бюджет,25.615385,В наличии,57,56,0,0
4822,https://hi-tech.md/kompyuternaya-tehnika/periferiynye-ustroystva/klaviatury-i-komplekty/usb/klaviatura-sven-kb-g9700-usb/,Т-000047600,H_компьютерная_техника,H_периферийные_устройства,H_клавиатуры,sven,Клавиатура Sven KB-G9700 USB,2024-08-06,499.0,999.0,1,50.05,бюджет,27.261364,В наличии,66,63,0,0
344,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/obogrevateli/konvektor-comfee-cendk15-15mrbk/,Т-000080313,H_бытовая_техника,H_климатическая_техника,H_обогреватели,другой,конвектор Comfee CENDK15-15MRBK,2024-08-06,1199.0,2399.0,1,50.02,бюджет,24.575419,В наличии,98,98,0,0
7211,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-5pl-40w-e27-480h480h260-06712-5-gl/,Т-000074390,H_товары_для_дома,H_освещение,H_люстры,другой,Люстра 5PL 40W E27 (480х480х260) (06712/5) (GL),2024-08-06,249.0,498.0,1,50.00,бюджет,39.909091,В наличии,197,197,0,0
5593,https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/rashodnye-materialy/burovoe-zubilo-deko-dkh0701-02-14-250mm/,Т-000087943,H_стройка_и_ремонт,H_электроинструменты,H_расходные_материалы,другой,Буровое зубило Deko DKH0701-02 (14/250мм),2024-08-06,21.0,42.0,1,50.00,бюджет,1.250000,В наличии,4,4,0,0
2212,https://hi-tech.md/bytovaya-tehnika/tehnika-karcher/komplektuyuschie-i-soputstvuyuschie-tovary-karcher/trubka-karcher-6.402-035.0-dlya-sv1802-1902/,Т-000006711,H_бытовая_техника,H_техника_karcher,H_комплектующие_и_сопутствующие_товары_karcher,karcher,трубка Karcher 6.402-035.0 (для SV1802/1902),2024-08-06,170.0,340.0,1,50.00,элитный,3.685185,В наличии,17,17,0,0
5911,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/bat.-panasonic-cr-1025el-1b/,Т-000037889,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,panasonic,Батарейка Panasonic CR-1025EL/1B,2024-08-06,9.0,18.0,1,50.00,бюджет,199.750000,В наличии,76,76,0,0
5912,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/bat.-panasonic-cr-1216el-1b/,Т-000037892,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,panasonic,Батарейка Panasonic CR-1216EL/1B,2024-08-06,9.0,18.0,1,50.00,бюджет,199.750000,В наличии,76,76,0,0
5924,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/bat.-panasonic-sr-721el-1b/,Т-000044640,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,panasonic,Батарейка Panasonic SR-721EL/1B,2024-08-06,17.0,34.0,1,50.00,бюджет,199.750000,В наличии,76,76,0,0
1988,https://hi-tech.md/bytovaya-tehnika/pylesosy-i-aksessuary/filtry-i-meshki-dlya-pyli/zaschitnaya-setka-zelmer-hepa-6012010111-dlya-5500-01z010/,Т-000022317,H_бытовая_техника,H_пылесосы_и_аксессуары,H_фильтры_и_мешки_для_пыли,zelmer,"защитная сетка Zelmer Hepa 6012010111(для 5500,01Z010)",2024-08-06,51.0,102.0,1,50.00,бюджет,12.882353,В наличии,41,41,0,0
